<a href="https://www.kaggle.com/code/biminhco/ogm-ge-crema-d?scriptVersionId=201724014" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/GeWu-Lab/OGM-GE_CVPR2022.git

Cloning into 'OGM-GE_CVPR2022'...
remote: Enumerating objects: 643, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 643 (delta 113), reused 166 (delta 91), pack-reused 436 (from 1)
Receiving objects: 100% (643/643), 332.42 MiB | 41.21 MiB/s, done.
Resolving deltas: 100% (345/345), done.


In [3]:
%cd /kaggle/working/OGM-GE_CVPR2022

/kaggle/working/OGM-GE_CVPR2022


In [4]:
!git clone https://github.com/CheyneyComputerScience/CREMA-D.git

Cloning into 'CREMA-D'...
remote: Enumerating objects: 22501, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 22501 (delta 22), reused 57 (delta 13), pack-reused 22428 (from 1)
Receiving objects: 100% (22501/22501), 14.82 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Updating files: 100% (22342/22342), done.
Filtering content: 100% (22326/22326), 3.42 GiB | 6.49 MiB/s, done.


In [5]:
%%writefile /kaggle/working/OGM-GE_CVPR2022/data/CREMAD/video_preprocessing.py
import pandas as pd
import cv2
import os
import pdb

class videoReader(object):
    def __init__(self, video_path, frame_interval=1, frame_kept_per_second=1):
        self.video_path = video_path
        self.frame_interval = frame_interval
        self.frame_kept_per_second = frame_kept_per_second

        #pdb.set_trace()
        self.vid = cv2.VideoCapture(self.video_path)
        self.fps = int(self.vid.get(cv2.CAP_PROP_FPS))
        self.video_frames = self.vid.get(cv2.CAP_PROP_FRAME_COUNT)
        self.video_len = int(self.video_frames/self.fps)


    def video2frame(self, frame_save_path):
        self.frame_save_path = frame_save_path
        success, image = self.vid.read()
        count = 0
        while success:
            count +=1
            if count % self.frame_interval == 0:
                save_name = '{}/frame_{}_{}.jpg'.format(self.frame_save_path, int(count/self.fps), count)  # filename_second_index
                cv2.imencode('.jpg', image)[1].tofile(save_name)
            success, image = self.vid.read()


    def video2frame_update(self, frame_save_path):
        self.frame_save_path = frame_save_path

        count = 0
        frame_interval = int(self.fps/self.frame_kept_per_second)
        while(count < self.video_frames):
            ret, image = self.vid.read()
            if not ret:
                break
            if count % self.fps == 0:
                frame_id = 0
            if frame_id<frame_interval*self.frame_kept_per_second and frame_id%frame_interval == 0:
                save_name = '{0}/{1:05d}.jpg'.format(self.frame_save_path, count)
                cv2.imencode('.jpg', image)[1].tofile(save_name)

            frame_id += 1
            count += 1


class CRAMED_dataset(object):
    def __init__(self, path_to_dataset = '/kaggle/working/OGM-GE_CVPR2022/data/CREMAD', frame_interval=1, frame_kept_per_second=1):
        self.path_to_video = os.path.join(path_to_dataset, 'VideoFlash')
        self.path_to_audio = os.path.join(path_to_dataset, 'AudioWAV')
        self.frame_kept_per_second = frame_kept_per_second
        self.path_to_save = os.path.join(path_to_dataset, 'Image-{:02d}-FPS'.format(self.frame_kept_per_second))
        if not os.path.exists(self.path_to_save):
            os.mkdir(self.path_to_save)

        csv_file = pd.read_csv(os.path.join(path_to_dataset, 'processedResults/summaryTable.csv'))
        self.file_list = list(csv_file['FileName'])

    def extractImage(self):

        for each_video in self.file_list:
            print('Precessing {} ...'.format(each_video))
            video_dir = os.path.join(self.path_to_video, each_video+'.flv')
            self.videoReader = videoReader(video_path=video_dir, frame_kept_per_second=self.frame_kept_per_second)

            save_dir = os.path.join(self.path_to_save, each_video)
            if not os.path.exists(save_dir):
                os.mkdir(save_dir)
            self.videoReader.video2frame_update(frame_save_path=save_dir)


cramed = CRAMED_dataset()
cramed.extractImage()

Overwriting /kaggle/working/OGM-GE_CVPR2022/data/CREMAD/video_preprocessing.py


In [6]:
!mv '/kaggle/working/OGM-GE_CVPR2022/CREMA-D/VideoFlash' '/kaggle/working/OGM-GE_CVPR2022/data/CREMAD'
!mv '/kaggle/working/OGM-GE_CVPR2022/CREMA-D/AudioWAV' '/kaggle/working/OGM-GE_CVPR2022/data/CREMAD'
!mv '/kaggle/working/OGM-GE_CVPR2022/CREMA-D/processedResults' '/kaggle/working/OGM-GE_CVPR2022/data/CREMAD'

In [7]:
%cd /kaggle/working/OGM-GE_CVPR2022/data/CREMAD
!python3 video_preprocessing.py

/kaggle/working/OGM-GE_CVPR2022/data/CREMAD
Precessing 1001_IEO_NEU_XX ...
Precessing 1001_IEO_HAP_LO ...
Precessing 1001_IEO_HAP_MD ...
Precessing 1001_IEO_HAP_HI ...
Precessing 1001_IEO_SAD_LO ...
Precessing 1001_IEO_SAD_MD ...
Precessing 1001_IEO_SAD_HI ...
Precessing 1001_IEO_ANG_LO ...
Precessing 1001_IEO_ANG_MD ...
Precessing 1001_IEO_ANG_HI ...
Precessing 1001_IEO_FEA_LO ...
Precessing 1001_IEO_FEA_MD ...
Precessing 1001_IEO_FEA_HI ...
Precessing 1001_IEO_DIS_LO ...
Precessing 1001_IEO_DIS_MD ...
Precessing 1001_IEO_DIS_HI ...
Precessing 1001_TIE_NEU_XX ...
Precessing 1001_TIE_HAP_XX ...
Precessing 1001_TIE_SAD_XX ...
Precessing 1001_TIE_ANG_XX ...
Precessing 1001_TIE_FEA_XX ...
Precessing 1001_TIE_DIS_XX ...
Precessing 1001_IOM_NEU_XX ...
Precessing 1001_IOM_HAP_XX ...
Precessing 1001_IOM_SAD_XX ...
Precessing 1001_IOM_ANG_XX ...
Precessing 1001_IOM_FEA_XX ...
Precessing 1001_IOM_DIS_XX ...
Precessing 1001_IWW_NEU_XX ...
Precessing 1001_IWW_HAP_XX ...
Precessing 1001_IWW_SAD_XX

In [8]:
%cd /kaggle/working/OGM-GE_CVPR2022

/kaggle/working/OGM-GE_CVPR2022


In [9]:
# !rm -rf /kaggle/working/OGM-GE_CVPR2022/saved_path

In [10]:
!mkdir saved_path

In [11]:
%%writefile /kaggle/working/OGM-GE_CVPR2022/main.py
import argparse
import os

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import pdb

from dataset.CramedDataset import CramedDataset
from dataset.VGGSoundDataset import VGGSound
from dataset.dataset import AVDataset
from models.basic_model import AVClassifier
from utils.utils import setup_seed, weight_init


def get_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', default='CREMAD', type=str,
                        help='VGGSound, KineticSound, CREMAD, AVE')
    parser.add_argument('--modulation', default='OGM_GE', type=str,

                        choices=['Normal', 'OGM', 'OGM_GE'])
    parser.add_argument('--fusion_method', default='concat', type=str,
                        choices=['sum', 'concat', 'gated', 'film'])
    parser.add_argument('--fps', default=1, type=int)
    parser.add_argument('--use_video_frames', default=3, type=int)
    parser.add_argument('--audio_path', default='/kaggle/working/OGM-GE_CVPR2022/data/CREMAD/AudioWAV', type=str)
    parser.add_argument('--visual_path', default='/kaggle/working/OGM-GE_CVPR2022/data/CREMAD', type=str)

    parser.add_argument('--batch_size', default=64, type=int)
    parser.add_argument('--epochs', default=100, type=int)

    parser.add_argument('--optimizer', default='sgd', type=str, choices=['sgd', 'adam'])
    parser.add_argument('--learning_rate', default=0.001, type=float, help='initial learning rate')
    parser.add_argument('--lr_decay_step', default=70, type=int, help='where learning rate decays')
    parser.add_argument('--lr_decay_ratio', default=0.1, type=float, help='decay coefficient')

    parser.add_argument('--modulation_starts', default=0, type=int, help='where modulation begins')
    parser.add_argument('--modulation_ends', default=50, type=int, help='where modulation ends')
    parser.add_argument('--alpha', required=True, type=float, help='alpha in OGM-GE')

    parser.add_argument('--ckpt_path', required=True, type=str, help='path to save trained models')
    parser.add_argument('--train', action='store_true', help='turn on train mode')

    parser.add_argument('--use_tensorboard', default=False, type=bool, help='whether to visualize')
    parser.add_argument('--tensorboard_path', type=str, help='path to save tensorboard logs')

    parser.add_argument('--random_seed', default=0, type=int)
    parser.add_argument('--gpu_ids', default='0, 1', type=str, help='GPU ids')

    return parser.parse_args()


def train_epoch(args, epoch, model, device, dataloader, optimizer, scheduler, writer=None):
    criterion = nn.CrossEntropyLoss()
    softmax = nn.Softmax(dim=1)
    relu = nn.ReLU(inplace=True)
    tanh = nn.Tanh()

    model.train()
    print("Start training ... ")

    _loss = 0
    _loss_a = 0
    _loss_v = 0

    for step, (spec, image, label) in enumerate(dataloader):

        #pdb.set_trace()
        spec = spec.to(device)
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        # TODO: make it simpler and easier to extend
        a, v, out = model(spec.unsqueeze(1).float(), image.float())

        if args.fusion_method == 'sum':
            out_v = (torch.mm(v, torch.transpose(model.module.fusion_module.fc_y.weight, 0, 1)) +
                     model.module.fusion_module.fc_y.bias)
            out_a = (torch.mm(a, torch.transpose(model.module.fusion_module.fc_x.weight, 0, 1)) +
                     model.module.fusion_module.fc_x.bias)
        else:
            weight_size = model.module.fusion_module.fc_out.weight.size(1)
            out_v = (torch.mm(v, torch.transpose(model.module.fusion_module.fc_out.weight[:, weight_size // 2:], 0, 1))
                     + model.module.fusion_module.fc_out.bias / 2)

            out_a = (torch.mm(a, torch.transpose(model.module.fusion_module.fc_out.weight[:, :weight_size // 2], 0, 1))
                     + model.module.fusion_module.fc_out.bias / 2)

        loss = criterion(out, label)
        loss_v = criterion(out_v, label)
        loss_a = criterion(out_a, label)
        loss.backward()

        if args.modulation == 'Normal':
            # no modulation, regular optimization
            pass
        else:
            # Modulation starts here !
            score_v = sum([softmax(out_v)[i][label[i]] for i in range(out_v.size(0))])
            score_a = sum([softmax(out_a)[i][label[i]] for i in range(out_a.size(0))])

            ratio_v = score_v / score_a
            ratio_a = 1 / ratio_v

            """
            Below is the Eq.(10) in our CVPR paper:
                    1 - tanh(alpha * rho_t_u), if rho_t_u > 1
            k_t_u =
                    1,                         else
            coeff_u is k_t_u, where t means iteration steps and u is modality indicator, either a or v.
            """

            if ratio_v > 1:
                coeff_v = 1 - tanh(args.alpha * relu(ratio_v))
                coeff_a = 1
            else:
                coeff_a = 1 - tanh(args.alpha * relu(ratio_a))
                coeff_v = 1

            if args.use_tensorboard:
                iteration = epoch * len(dataloader) + step
                writer.add_scalar('data/ratio v', ratio_v, iteration)
                writer.add_scalar('data/coefficient v', coeff_v, iteration)
                writer.add_scalar('data/coefficient a', coeff_a, iteration)

            if args.modulation_starts <= epoch <= args.modulation_ends: # bug fixed
                for name, parms in model.named_parameters():
                    layer = str(name).split('.')[1]

                    if 'audio' in layer and len(parms.grad.size()) == 4:
                        if args.modulation == 'OGM_GE':  # bug fixed
                            parms.grad = parms.grad * coeff_a + \
                                         torch.zeros_like(parms.grad).normal_(0, parms.grad.std().item() + 1e-8)
                        elif args.modulation == 'OGM':
                            parms.grad *= coeff_a

                    if 'visual' in layer and len(parms.grad.size()) == 4:
                        if args.modulation == 'OGM_GE':  # bug fixed
                            parms.grad = parms.grad * coeff_v + \
                                         torch.zeros_like(parms.grad).normal_(0, parms.grad.std().item() + 1e-8)
                        elif args.modulation == 'OGM':
                            parms.grad *= coeff_v
            else:
                pass


        optimizer.step()

        _loss += loss.item()
        _loss_a += loss_a.item()
        _loss_v += loss_v.item()

    scheduler.step()

    return _loss / len(dataloader), _loss_a / len(dataloader), _loss_v / len(dataloader)


def valid(args, model, device, dataloader):
    softmax = nn.Softmax(dim=1)

    if args.dataset == 'VGGSound':
        n_classes = 309
    elif args.dataset == 'KineticSound':
        n_classes = 31
    elif args.dataset == 'CREMAD':
        n_classes = 6
    elif args.dataset == 'AVE':
        n_classes = 28
    else:
        raise NotImplementedError('Incorrect dataset name {}'.format(args.dataset))

    with torch.no_grad():
        model.eval()
        # TODO: more flexible
        num = [0.0 for _ in range(n_classes)]
        acc = [0.0 for _ in range(n_classes)]
        acc_a = [0.0 for _ in range(n_classes)]
        acc_v = [0.0 for _ in range(n_classes)]

        for step, (spec, image, label) in enumerate(dataloader):

            spec = spec.to(device)
            image = image.to(device)
            label = label.to(device)

            a, v, out = model(spec.unsqueeze(1).float(), image.float())

            if args.fusion_method == 'sum':
                out_v = (torch.mm(v, torch.transpose(model.module.fusion_module.fc_y.weight, 0, 1)) +
                         model.module.fusion_module.fc_y.bias / 2)
                out_a = (torch.mm(a, torch.transpose(model.module.fusion_module.fc_x.weight, 0, 1)) +
                         model.module.fusion_module.fc_x.bias / 2)
            else:
                out_v = (torch.mm(v, torch.transpose(model.module.fusion_module.fc_out.weight[:, 512:], 0, 1)) +
                         model.module.fusion_module.fc_out.bias / 2)
                out_a = (torch.mm(a, torch.transpose(model.module.fusion_module.fc_out.weight[:, :512], 0, 1)) +
                         model.module.fusion_module.fc_out.bias / 2)

            prediction = softmax(out)
            pred_v = softmax(out_v)
            pred_a = softmax(out_a)

            for i in range(image.shape[0]):

                ma = np.argmax(prediction[i].cpu().data.numpy())
                v = np.argmax(pred_v[i].cpu().data.numpy())
                a = np.argmax(pred_a[i].cpu().data.numpy())
                num[label[i]] += 1.0

                #pdb.set_trace()
                if np.asarray(label[i].cpu()) == ma:
                    acc[label[i]] += 1.0
                if np.asarray(label[i].cpu()) == v:
                    acc_v[label[i]] += 1.0
                if np.asarray(label[i].cpu()) == a:
                    acc_a[label[i]] += 1.0

    return sum(acc) / sum(num), sum(acc_a) / sum(num), sum(acc_v) / sum(num)


def main():
    args = get_arguments()
    print(args)

    setup_seed(args.random_seed)
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_ids
    gpu_ids = list(range(torch.cuda.device_count()))

    device = torch.device('cuda:0')

    model = AVClassifier(args)

    model.apply(weight_init)
    model.to(device)

    model = torch.nn.DataParallel(model, device_ids=gpu_ids)

    model.cuda()

    optimizer = optim.SGD(model.parameters(), lr=args.learning_rate, momentum=0.9, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, args.lr_decay_step, args.lr_decay_ratio)

    if args.dataset == 'VGGSound':
        train_dataset = VGGSound(args, mode='train')
        test_dataset = VGGSound(args, mode='test')
    elif args.dataset == 'KineticSound':
        train_dataset = AVDataset(args, mode='train')
        test_dataset = AVDataset(args, mode='test')
    elif args.dataset == 'CREMAD':
        train_dataset = CramedDataset(args, mode='train')
        test_dataset = CramedDataset(args, mode='test')
    elif args.dataset == 'AVE':
        train_dataset = AVDataset(args, mode='train')
        test_dataset = AVDataset(args, mode='test')
    else:
        raise NotImplementedError('Incorrect dataset name {}! '
                                  'Only support VGGSound, KineticSound and CREMA-D for now!'.format(args.dataset))

    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size,
                                  shuffle=True, num_workers=32, pin_memory=True)

    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size,
                                 shuffle=False, num_workers=32, pin_memory=True)

    if args.train:

        best_acc = 0.0

        for epoch in range(args.epochs):

            print('Epoch: {}: '.format(epoch))

            if args.use_tensorboard:

                writer_path = os.path.join(args.tensorboard_path, args.dataset)
                if not os.path.exists(writer_path):
                    os.mkdir(writer_path)
                log_name = '{}_{}'.format(args.fusion_method, args.modulation)
                writer = SummaryWriter(os.path.join(writer_path, log_name))

                batch_loss, batch_loss_a, batch_loss_v = train_epoch(args, epoch, model, device,
                                                                     train_dataloader, optimizer, scheduler)
                acc, acc_a, acc_v = valid(args, model, device, test_dataloader)

                writer.add_scalars('Loss', {'Total Loss': batch_loss,
                                            'Audio Loss': batch_loss_a,
                                            'Visual Loss': batch_loss_v}, epoch)

                writer.add_scalars('Evaluation', {'Total Accuracy': acc,
                                                  'Audio Accuracy': acc_a,
                                                  'Visual Accuracy': acc_v}, epoch)

            else:
                batch_loss, batch_loss_a, batch_loss_v = train_epoch(args, epoch, model, device,
                                                                     train_dataloader, optimizer, scheduler)
                acc, acc_a, acc_v = valid(args, model, device, test_dataloader)

            if acc > best_acc:
                best_acc = float(acc)

                if not os.path.exists(args.ckpt_path):
                    os.mkdir(args.ckpt_path)

                model_name = 'best_model_of_dataset_{}_{}_alpha_{}_' \
                             'optimizer_{}_modulate_starts_{}_ends_{}_' \
                             'epoch_{}_acc_{}.pth'.format(args.dataset,
                                                          args.modulation,
                                                          args.alpha,
                                                          args.optimizer,
                                                          args.modulation_starts,
                                                          args.modulation_ends,
                                                          epoch, acc)

                saved_dict = {'saved_epoch': epoch,
                              'modulation': args.modulation,
                              'alpha': args.alpha,
                              'fusion': args.fusion_method,
                              'acc': acc,
                              'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'scheduler': scheduler.state_dict()}

                save_dir = os.path.join(args.ckpt_path, model_name)

                torch.save(saved_dict, save_dir)
                print('The best model has been saved at {}.'.format(save_dir))
                print("Loss: {:.3f}, Acc: {:.3f}".format(batch_loss, acc))
                print("Audio Acc: {:.3f}， Visual Acc: {:.3f} ".format(acc_a, acc_v))
            else:
                print("Loss: {:.3f}, Acc: {:.3f}, Best Acc: {:.3f}".format(batch_loss, acc, best_acc))
                print("Audio Acc: {:.3f}， Visual Acc: {:.3f} ".format(acc_a, acc_v))

    else:
        # first load trained model
        loaded_dict = torch.load(args.ckpt_path)
        # epoch = loaded_dict['saved_epoch']
        modulation = loaded_dict['modulation']
        # alpha = loaded_dict['alpha']
        fusion = loaded_dict['fusion']
        state_dict = loaded_dict['model']
        # optimizer_dict = loaded_dict['optimizer']
        # scheduler = loaded_dict['scheduler']

        assert modulation == args.modulation, 'inconsistency between modulation method of loaded model and args !'
        assert fusion == args.fusion_method, 'inconsistency between fusion method of loaded model and args !'

        model = model.load_state_dict(state_dict)
        print('Trained model loaded!')

        acc, acc_a, acc_v = valid(args, model, device, test_dataloader)
        print('Accuracy: {}, accuracy_a: {}, accuracy_v: {}'.format(acc, acc_a, acc_v))


if __name__ == "__main__":
    main()


Overwriting /kaggle/working/OGM-GE_CVPR2022/main.py


In [12]:
%cd /kaggle/working/OGM-GE_CVPR2022

/kaggle/working/OGM-GE_CVPR2022


In [13]:
!python main.py --alpha 0.3 --ckpt_path '/kaggle/working/OGM-GE_CVPR2022/saved_path' --train 

Namespace(dataset='CREMAD', modulation='OGM_GE', fusion_method='concat', fps=1, use_video_frames=3, audio_path='/kaggle/working/OGM-GE_CVPR2022/data/CREMAD/AudioWAV', visual_path='/kaggle/working/OGM-GE_CVPR2022/data/CREMAD', batch_size=64, epochs=100, optimizer='sgd', learning_rate=0.001, lr_decay_step=70, lr_decay_ratio=0.1, modulation_starts=0, modulation_ends=50, alpha=0.3, ckpt_path='/kaggle/working/OGM-GE_CVPR2022/saved_path', train=True, use_tensorboard=False, tensorboard_path=None, random_seed=0, gpu_ids='0, 1')
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg

In [14]:
!zip -r /kaggle/working/best_model.zip /kaggle/working/OGM-GE_CVPR2022/saved_path

  adding: kaggle/working/OGM-GE_CVPR2022/saved_path/ (stored 0%)
  adding: kaggle/working/OGM-GE_CVPR2022/saved_path/best_model_of_dataset_CREMAD_OGM_GE_alpha_0.3_optimizer_sgd_modulate_starts_0_ends_50_epoch_47_acc_0.6115591397849462.pth (deflated 7%)
  adding: kaggle/working/OGM-GE_CVPR2022/saved_path/best_model_of_dataset_CREMAD_OGM_GE_alpha_0.3_optimizer_sgd_modulate_starts_0_ends_50_epoch_3_acc_0.45026881720430106.pth (deflated 7%)
  adding: kaggle/working/OGM-GE_CVPR2022/saved_path/best_model_of_dataset_CREMAD_OGM_GE_alpha_0.3_optimizer_sgd_modulate_starts_0_ends_50_epoch_69_acc_0.6370967741935484.pth (deflated 7%)
  adding: kaggle/working/OGM-GE_CVPR2022/saved_path/best_model_of_dataset_CREMAD_OGM_GE_alpha_0.3_optimizer_sgd_modulate_starts_0_ends_50_epoch_70_acc_0.6411290322580645.pth (deflated 7%)
  adding: kaggle/working/OGM-GE_CVPR2022/saved_path/best_model_of_dataset_CREMAD_OGM_GE_alpha_0.3_optimizer_sgd_modulate_starts_0_ends_50_epoch_31_acc_0.5940860215053764.pth (deflated

In [15]:
%cd /kaggle/working
!ls

/kaggle/working
OGM-GE_CVPR2022  __notebook__.ipynb  best_model.zip


In [16]:
from IPython.display import FileLink 
FileLink(r'best_model.zip')

/kaggle/working/best_model.zip